# Nexus Quick Start
## iQuHACK 2025

Welcome to the Quantinuum Nexus Lab! A hosted Jupyter environment with the pre-installed Python software packages you'll need, including:

* [`pytket`](https://tket.quantinuum.com/api-docs/index.html): Quantinuum's Python SDK for TKET quantum programming tools.
* [`qnexus`](https://docs.quantinuum.com/nexus/): the client software for submitting jobs to the Quantinuum Nexus platform

You can also open a terminal in the Lab to commit and push your changes to your team's fork of this repository.

### Step 1. Enable access to the Quantinuum simulator and emulator

We provide two systems for you to target when executing quantum circuits as part of the challenge.



* `H1-Emulator` (recommmended): A noisy of Quantinuum's 20 qubit `H1-1` quantum computer hosted within Quantinuum Nexus.
* `H1-1LE`: a noiseless simulator of Quantinuum's 20 qubit `H1-1` quantum computer.
* `H1-1E`: a noisy 'emulator' of `H1-1`. This system is much closer to the actual `H1-1` machine, and has longer queuing times. We recommend only using this system when you need to execute your circuit on something closer to the real hardware.
* `H1-1SC`: a 'syntax checker' system for optionally testing things out (it won't run any code but can be used for validation of your usage of the API). 

In order to access these systems, you'll need to activate your access:

* Visit https://nexus.quantinuum.com/backends/quantinuum and enter your username and password credentials by selecting 'Configure'.
* You'll know if this succeeds as your available Quantinuum devices will be displayed below. The `H1-1LE` simulator is very new and might not be displayed, so don't worry if it doesn't show up.


### Step 2: Try out the example code below to compile and simulate your first circuit.

- Copy this file or its contents into the Quantinuum Nexus hub and run the code below to see some example usage.


## Lets compose a quantum circuit using Pytket

Nexus uses [pytket Circuits](https://cqcl.github.io/pytket/manual/manual_circuit.html). Here we'll create a simple one.

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as draw

circuit = Circuit(4, 4).H(0).CX(0, 1).CX(1, 2).CX(2, 3)
circuit.measure_all()

circuit.name = "my-first-circuit"

draw(circuit)

### Create a Nexus Project

A project is a way to compartmentalize your running jobs and data, allowing you to retrieve and share data easily.

In [9]:
import qnexus as qnx

# ADD YOUR OWN PROJECT NAME HERE
project = qnx.projects.get_or_create(name="iQuHACK2025-demo")
qnx.context.set_active_project(project)

You should be able to see this project if you visit [https://nexus.quantinuum.com/projects/all](https://nexus.quantinuum.com/projects/all)

In [10]:
import datetime

jobname_suffix = datetime.datetime.now().strftime("%Y_%m_%d-%H-%M-%S")

### Set up your Backend Configuration

In Quantinuum Nexus, we use `BackendConfig`s to specify the details of the quantum system we want to target.


For this example, we'll use ``QuantinuumConfig``, to configure Nexus to target a simulator for Quantinuum's `H1-1LE` device. 

We'll pass in the `device_name` and also `user_group="iQuHACK_2024"` to identify jobs as part of iQuHack 2024.

In [11]:
h1_config = qnx.QuantinuumConfig(device_name="H1-Emulator")

aer_config = qnx.AerStateConfig()

## Upload and download circuits

In [12]:
ref = qnx.circuits.upload(circuit=circuit, name=f"GHZ-Circuit-{jobname_suffix}")

In [ ]:
draw(ref.download_circuit())

### Compile the circuit

Nexus offers compilation with the TKET quantum compiler, this is an easy way to:

* Optimize the circuit
* Convert it to run on a specific quantum device, which might not support all quantum gates.

For the challenge you can use pytket directly to compile circuits, or you can get Nexus to do the compilation for you. If you'd like to disable any changes to your circuit other than making sure that the gateset is correct, you can always set `optimisation_level=0`.

In [ ]:
ref_compile_job = qnx.start_compile_job(
    circuits=[ref],
    backend_config=aer_config,
    optimisation_level=2,
    name=f"compilation-job-{jobname_suffix}"
)

You should now see the compile show up when you click on your project on [https://nexus.quantinuum.com/projects/all](https://nexus.quantinuum.com/projects/all).

If you're curious to see what the compiler did, you can click on the job results and see all the passes the compiler performed on your circuit. 

In [15]:
qnx.jobs.wait_for(ref_compile_job)
ref_compiled_circuit = qnx.jobs.results(ref_compile_job)[0].get_output()

In [ ]:
compiled_circuit = ref_compiled_circuit.download_circuit()

draw(compiled_circuit)

### Submit the circuit for execution

Nexus will take care of submitting the circuit to your chosen device and storing the results when it has executed.

In this case we are executing the circuit on a simulator running in the Nexus cluster so we should get the results back quickly, however many quantum devices have lengthy queues in which case Nexus will track the job and retrieve the results when they are ready.

In [17]:
ref_execute_job = qnx.start_execute_job(
    circuits=[ref_compiled_circuit],
    n_shots=[100],
    backend_config=aer_config,
    name=f"execution-job-{jobname_suffix}"
)

In [ ]:
qnx.jobs.status(ref_execute_job)

In [19]:
qnx.jobs.wait_for(ref_execute_job)
ref_result = qnx.jobs.results(ref_execute_job)[0]
backend_result = ref_result.download_result()

In [ ]:
print(backend_result.get_counts())

You can also view the job from the project page on the Quantinuum Nexus website and check out the results there.


### Next steps:

Use the website to:
- make a `Team` on the Nexus website and add your hackathon team members.
- give this `Team` access to the Nexus `Project` you just created.